In [34]:
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram, stft, istft
import matplotlib.pyplot as plt
import librosa
import librosa.display
from misceallaneous import getWavFileAsNpArray, displaySpectrogram
from IPython.display import Audio

samplerate = 12000
nperseg = 1024

clean = getWavFileAsNpArray("../dataset_2/clean/p1.wav")
white = getWavFileAsNpArray("../dataset_2/white/p1.wav")
clean = np.array(clean, dtype="int32")
white = np.array(white, dtype="int32")

clean_split = []
white_split = []

samples_length = nperseg

for i in range(0, clean.shape[0]-samples_length, samples_length):
    clean_split.append(clean[i:i+samples_length])
    white_split.append(white[i:i+samples_length])
clean_split = np.array(clean_split)
white_split = np.array(white_split)

print(clean_split.shape)
print(white_split.shape)

(10659, 1024)
(10659, 1024)


In [35]:
input_test0 = np.reshape(clean_split[0, :], (1, clean_split.shape[1]))
input_test1 = np.reshape(clean_split[1, :], (1, clean_split.shape[1]))
input_test2 = np.reshape(clean_split[2, :], (1, clean_split.shape[1]))
b1 = np.reshape(input_test0.T, (clean_split.shape[1],))
b2 = np.reshape(input_test1.T, (clean_split.shape[1],))
b3 = np.reshape(input_test2.T, (clean_split.shape[1],))
b = np.concatenate((b1, b2, b3))

c, t, Vxx_input_test = stft(b, fs=samplerate, nperseg=nperseg)
displaySpectrogram(Vxx_input_test)
plt.show()

In [36]:
def process_stft(inputs):
    return tf.signal.stft(inputs, frame_length=nperseg, frame_step=nperseg)

In [37]:
def generator(size):
    inputs = tf.keras.Input(shape=(size,))
    x1 = tf.keras.layers.Dense(size, activation="relu")(inputs)
    x2 = tf.keras.layers.Dense(int(size*0.5), activation="relu")(x1)
    x3 = tf.keras.layers.Dense(int(size*0.2), activation="relu")(x2)
    x4 = tf.keras.layers.Dense(int(size*0.5), activation="relu")(x3)
    outputs = tf.keras.layers.Dense(size, activation="relu")(x4)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="generator")
    model.summary()
    model.compile(optimizer='adam', loss='bce', metrics=['accuracy'])
    return model

def discriminator(size):
    inputs = tf.keras.Input(shape=(size,))
    x0 = tf.keras.layers.Dropout(0.1)(inputs)
    x1 = tf.keras.layers.Dense(int(size), activation="relu")(x0)
    x01 = tf.keras.layers.Dropout(0.1)(x1)
    x2 = tf.keras.layers.Dense(int(size*0.7), activation="relu")(x1)
    x02 = tf.keras.layers.Dropout(0.1)(x2)
    x3 = tf.keras.layers.Dense(int(size*0.5), activation="tanh")(x2)
    outputs = tf.keras.layers.Dense(2, activation="sigmoid")(x3)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="discriminator")
    model.summary()
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

In [43]:
def skip_generator(size):
    inputs = tf.keras.Input(shape=(size,))
    x1 = tf.keras.layers.Dense(int(size*0.5), activation="relu")(inputs)
    x2 = tf.keras.layers.Dense(int(size*0.1), activation="relu")(x1)
    x3 = tf.keras.layers.Dense(int(size*0.5), activation="relu")(x2)
    x4 = tf.keras.layers.Add()([x1, x3])
    x5 = tf.keras.layers.Dense(size, activation="relu")(x4)
    outputs = tf.keras.layers.Add()([inputs, x5])
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="skip_autoencoder")
    model.summary()
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

def stft_discriminator(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x0 = tf.keras.layers.Lambda(process_stft)(inputs)
    x1 = tf.keras.layers.Dense(512, activation="tanh")(x0)
    x2 = tf.keras.layers.Dense(256, activation="tanh")(x1)
    x3 = tf.keras.layers.Dense(128, activation="tanh")(x2)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x3)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="discriminator")
    model.summary()
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model


def GAN(size, g, d):
    z = tf.keras.layers.Input(shape=(size,))
    image = g(z)
    valid = d(image)
    
    combined_network = tf.keras.Model(z, valid)
    combined_network.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return combined_network

g = skip_generator(clean_split.shape[1])
d = stft_discriminator(clean_split.shape[1])

Model: "skip_autoencoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 512)          524800      input_33[0][0]                   
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 102)          52326       dense_8[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 512)          52736       dense_9[0][0]                    
___________________________________________________________________________________

In [44]:
def train_discriminator(g, d, white, clean, white_labels, clean_labels, train_size=clean.shape[0], accuracy=0.45):
    g_outputs = []
    print("Predicting Generator Outputs")
    for i in range(train_size):
        t = np.reshape(white_split[i, :], (1, white_split.shape[1]))
        o = np.reshape(np.array(g.predict(t)), nperseg)
        g_outputs.append(o)
    g_outputs = np.array(g_outputs)
    labels = -1*np.ones((train_size, 1))
    history = d.fit(np.concatenate((g_outputs, clean[:train_size,])), np.concatenate((labels, clean_labels[:train_size,])))
    while history.history['accuracy'][-1] < accuracy:
        history = d.fit(np.concatenate((g_outputs, clean[:train_size,])), np.concatenate((labels, clean_labels[:train_size,])))
    return history, d

In [45]:
clean_labels = np.ones((clean_split.shape[0], 1))
white_labels = np.zeros((white_split.shape[0], 1))

In [46]:
print("Pre-training the generator")
history = g.fit(white_split, clean_split)
while history.history['accuracy'][-1] < 0.4:
    history = g.fit(white_split, clean_split)

Pre-training the generator


RuntimeError: Detected a call to `Model.fit` inside a `tf.function`. `Model.fit is a high-level endpoint that manages its own `tf.function`. Please move the call to `Model.fit` outside of all enclosing `tf.function`s. Note that you can call a `Model` directly on `Tensor`s inside a `tf.function` like: `model(x)`.

In [42]:
history, d = train_discriminator(g, d, white_split, clean_split, white_labels, clean_labels, 5000)
d.compile(optimizer= tf.keras.optimizers.Adam(lr=0.01), loss='mse', metrics=['accuracy'])
for i in range(20):
    d.trainable = False
    g.trainable = True
    gan = GAN(clean_split.shape[1], g, d)
    print("Training the generator")
    history = gan.fit(white_split, clean_labels, epochs=15)
    
    d.trainable = True
    g.trainable = False
    print("Training the discriminator")
    #history = d.fit(np.concatenate((white_split, clean_split)), np.concatenate((white_labels, clean_labels)), epochs=3)
    history, d = train_discriminator(g, d, white_split, clean_split, white_labels, clean_labels, 5000)

Predicting Generator Outputs


RuntimeError: Detected a call to `Model.predict` inside a `tf.function`. `Model.predict is a high-level endpoint that manages its own `tf.function`. Please move the call to `Model.predict` outside of all enclosing `tf.function`s. Note that you can call a `Model` directly on `Tensor`s inside a `tf.function` like: `model(x)`.

In [18]:
output_test0 = g.predict(input_test0)
output_test1 = g.predict(input_test1)
output_test2 = g.predict(input_test2)
o1 = np.reshape(output_test0.T, (white_split.shape[1],))
o2 = np.reshape(output_test1.T, (white_split.shape[1],))
o3 = np.reshape(output_test2.T, (white_split.shape[1],))
o = np.concatenate((o1, o2, o3))
c, t, Vxx_test = stft(o, fs=samplerate, nperseg=nperseg)
displaySpectrogram(Vxx_test)
plt.show()

In [19]:
i = [np.reshape(white_split[k, :], (1, white_split.shape[1])) for k in range(25)]
o = [g.predict(i[k]) for k in range(len(i))]
p = [np.reshape(i[k].T, (white_split.shape[1],)) for k in range(len(o))]
q = [np.reshape(o[k].T, (white_split.shape[1],)) for k in range(len(o))]


a = np.concatenate(p)
b = np.concatenate(q)
Audio(a, rate=samplerate)

In [20]:
Audio(b, rate=samplerate)